In [1]:
# 사고데이터 수집
# 데이터 크롤링을  사용해 데이터셋을만들고, 이미지로 학습하는 모델
from icrawler.builtin import GoogleImageCrawler

google_crawler = GoogleImageCrawler(parser_threads=2, downloader_threads=4, storage={'root_dir': './data'})

google_crawler.crawl(keyword = 'car crash', max_num=500, min_size=(200, 200), max_size=(None))

2022-02-15 10:20:52,906 - INFO - icrawler.crawler - start crawling...
2022-02-15 10:20:52,906 - INFO - icrawler.crawler - starting 1 feeder threads...
2022-02-15 10:20:52,906 - INFO - feeder - thread feeder-001 exit
2022-02-15 10:20:52,906 - INFO - icrawler.crawler - starting 2 parser threads...
2022-02-15 10:20:52,914 - INFO - icrawler.crawler - starting 4 downloader threads...
2022-02-15 10:20:54,138 - INFO - parser - parsing result page https://www.google.com/search?q=car+crash&ijn=1&start=100&tbs=&tbm=isch
2022-02-15 10:20:54,169 - INFO - downloader - skip downloading file 000001.jpg
2022-02-15 10:20:54,169 - INFO - downloader - skip downloading file 000002.jpg
2022-02-15 10:20:54,184 - INFO - downloader - skip downloading file 000003.jpg
2022-02-15 10:20:54,184 - INFO - downloader - skip downloading file 000004.jpg
2022-02-15 10:20:54,184 - INFO - downloader - skip downloading file 000005.jpg
2022-02-15 10:20:54,184 - INFO - downloader - skip downloading file 000006.jpg
2022-02-15

2022-02-15 10:20:55,486 - ERROR - downloader - Response status code 403, file https://www.chicagotribune.com/resizer/_nmtVmAoiNKfblN0GD2487Cp6fQ\u003d/1200x0/top/cloudfront-us-east-1.images.arcpublishing.com/tronc/GNV5574VDZCDXMAPQH6UHNC6II.jpg
2022-02-15 10:20:55,805 - INFO - parser - parsing result page https://www.google.com/search?q=car+crash&ijn=4&start=400&tbs=&tbm=isch
2022-02-15 10:20:56,989 - INFO - parser - no more page urls for thread parser-001 to parse
2022-02-15 10:20:56,991 - INFO - parser - thread parser-001 exit
2022-02-15 10:20:57,291 - INFO - downloader - image #89	https://i2-prod.dailyrecord.co.uk/news/scottish-news/article26212003.ece/ALTERNATES/s1200c/0_Screen-Shot-2022-02-12-at-112352.png
2022-02-15 10:20:57,849 - INFO - parser - no more page urls for thread parser-002 to parse
2022-02-15 10:20:57,849 - INFO - parser - thread parser-002 exit
2022-02-15 10:21:00,224 - INFO - downloader - no more download task for thread downloader-002
2022-02-15 10:21:00,225 - INF

In [37]:
# 수집한 데이터를 이미지 처리 및 t/t set 으로 나누는 과정
from PIL import Image
import os, glob
import numpy as np
from sklearn.model_selection import train_test_split

# 분류 대상 카테고리 선택하기 
accident_dir = "./image"
categories = ["Car front crash","Car side crash","Rear and crash","Car broken windshield","Car scratch","Flat tire","Overturned vehicle"]
nb_classes = len(categories)

# 이미지 크기 지정 
image_w = 64 
image_h = 64
pixels = image_w * image_h * 3

# 이미지 데이터 읽어 들이기 
X = []
Y = []
for idx, cat in enumerate(categories):
    # 레이블 지정 
    label = [0 for i in range(nb_classes)]
    label[idx] = 1
    
    # 이미지 
    image_dir = accident_dir + "/" + cat
    #image_dir = (r"C:\anaconda\keras\data")
    files = glob.glob(image_dir+"/*.jpg")
    
    for i, f in enumerate(files):
        img = Image.open(f) 
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)      # numpy 배열로 변환
        X.append(data)
        Y.append(label)
        if i % 10 == 0:
            print(i, "\n", data)
        #print(f"{i}:{label}")
X = np.array(X)
Y = np.array(Y)
# 학습 전용 데이터와 테스트 전용 데이터 구분 
X_train, X_test, y_train, y_test = train_test_split(X, Y)
xy = (X_train, X_test, y_train, y_test)

print(">>> data 저장중...")
np.save("./image/7obj.npy", xy)
print("ok", len(Y))


0 
 [[[100 119 107]
  [ 98 118 106]
  [ 97 115 103]
  ...
  [171 166 163]
  [168 163 160]
  [171 167 164]]

 [[ 94 116 103]
  [ 97 117 105]
  [ 98 117 101]
  ...
  [171 166 163]
  [180 175 171]
  [169 165 160]]

 [[ 91 113  98]
  [ 94 117  97]
  [ 93 116  95]
  ...
  [168 163 158]
  [170 166 153]
  [172 168 159]]

 ...

 [[174 159 152]
  [161 149 137]
  [171 158 146]
  ...
  [ 93  93  99]
  [ 97  96  97]
  [ 76  77  81]]

 [[169 157 148]
  [166 156 150]
  [142 133 121]
  ...
  [142 146 161]
  [142 143 154]
  [127 130 150]]

 [[154 141 137]
  [158 144 140]
  [158 144 138]
  ...
  [133 134 151]
  [142 147 165]
  [143 154 175]]]
0 
 [[[ 34  34  34]
  [ 38  36  36]
  [ 75  58  43]
  ...
  [ 39  36  38]
  [ 52  50  52]
  [ 33  30  32]]

 [[ 33  33  33]
  [ 35  31  30]
  [ 51  45  40]
  ...
  [ 58  55  57]
  [ 57  55  57]
  [ 44  41  42]]

 [[ 35  35  34]
  [ 37  37  36]
  [ 41  41  41]
  ...
  [ 62  57  60]
  [ 53  49  51]
  [ 51  50  51]]

 ...

 [[149 141 136]
  [153 144 139]
  [157 147 1

In [39]:
# CNN Model (keras)

from keras.models import Sequential
from keras.layers import MaxPooling2D
from keras.layers import Conv2D
from keras.layers import Activation, Dropout, Flatten, Dense
import numpy as np
import os

#카테고리 
categories = ["Car front crash",
              "Car side crash",
              "Rear and crash",
              "Car broken windsheid",
              "Car scratch",
              "Flat tire",
              "Overturned vehicle"]

nb_classes = len(categories)

img_w = 64
img_h = 64
X_train, X_test, y_train, y_test = np.load('./image/7obj.npy', allow_pickle=True)

#데이터 정규화
X_train = X_train.astype("float")/256
X_test = X_test.astype('float')/256
print("X_train shape: ", X_train.shape)

# 모델 구조 정의
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:], padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), input_shape=X_train.shape[1:], padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# FC
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# 모델 구축, keras. compile
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# 모델 확인 
print(model.summary())

hdf5_file = "./7obj-model.hdf5"
if os.path.exists(hdf5_file):
    model.load_weights(hdf5_file)
else:
    model.fit(X_train, y_train, batch_size=32, epochs=10)
    model.save_weights(hdf5_file)

X_train shape:  (5, 64, 64, 3)
Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_45 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
activation_60 (Activation)   (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_45 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_61 (Activation)   (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_47 (Conv2D)    

In [40]:
score = model.evaluate(X_test, y_test)
print('loss=', score[0])
print('accuracy=', score[1])

1/1 [==============================] - 0s 109ms/step - loss: 3.2084 - accuracy: 0.5000
loss= 3.2084479331970215
accuracy= 0.5


In [41]:
#적용시킬 이밎 선택
test_image = './data/000001.jpg'
img = Image.open(test_image)
img = img.convert("RGB")
img = img.resize((64,64))
data = np.array(img)
X = np.array(data)
X = X.astype("float")/256
X = X.reshape(-1, 64, 64, 3)

# 예측
pred = model.predict(X)
result = [np.argmax(value) for value in pred]
print("Data's Category is :", categories[result[0]])

FileNotFoundError: [Errno 2] No such file or directory: './data/000001.jpg'